In [1]:
import pandas as pd
import numpy as np
import datetime
import time

**train_mfti.parquet** – сырые данные, которые можно использовать для обучения модели

* event_date – дата взаимодейтсвия
* event_timestamp – timestamp взаимодействия в секундах 
* vacancy_id_ - id вакансии, с которой было взаимодействие
* cookie_id – id пользователя по его браузеру/ip/устройству
* user_id – id пользователя на сайте rabota.ru (есть только для зарегистрированных пользователей)
* event_type – тип взаимодействия

In [3]:
#load dataset 
df = pd.read_parquet('Data/train_mfti.parquet')

In [4]:
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


In [5]:
#сортируем датасет по столбцу 'cookie_id', а потом по 'event_timestamp'
sorted_df = df.sort_values(['cookie_id','event_timestamp'])
#добавляем столбец "action_time" и заполняем 0
sorted_df.insert(2, "action_time", 0)
#добавляем новые индексы
sorted_df = sorted_df.reset_index()
sorted_df.head(10)

,index,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,4550690,2022-08-30,1661866548,0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,4653413,2022-08-30,1661866548,0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,4650401,2022-08-30,1661866559,0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,4693679,2022-08-30,1661866559,0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,4650417,2022-08-30,1661866564,0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
5,4693678,2022-08-30,1661866564,0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
6,4510912,2022-08-30,1661866577,0,174953,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
7,4649889,2022-08-30,1661866577,0,174953,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
8,11346982,2022-08-30,1661866594,0,174953,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,click_favorite
9,4511048,2022-08-30,1661866625,0,176171,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy


In [6]:
#получаем разницу по времени между двумя событиями 
sorted_df['action_time'] = sorted_df['event_timestamp'].diff(periods=1)
sorted_df[0:25]

,index,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,4550690,2022-08-30,1661866548,NaN,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,4653413,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,4650401,2022-08-30,1661866559,11.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,4693679,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,4650417,2022-08-30,1661866564,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
5,4693678,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
6,4510912,2022-08-30,1661866577,13.0,174953,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
7,4649889,2022-08-30,1661866577,0.0,174953,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
8,11346982,2022-08-30,1661866594,17.0,174953,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,click_favorite
9,4511048,2022-08-30,1661866625,31.0,176171,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy


In [7]:
sorted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   index            int64  
 1   event_date       object 
 2   event_timestamp  int64  
 3   action_time      float64
 4   vacancy_id_      int64  
 5   cookie_id        object 
 6   user_id          object 
 7   event_type       object 
dtypes: float64(1), int64(3), object(4)
memory usage: 750.3+ MB


In [8]:

#столбец 'action_time' type преобразуем в int32
#sorted_df.astype({'action_time': 'int32'}).dtypes
sorted_df['action_time'] = pd.to_numeric(sorted_df['action_time'], errors='coerce').fillna(0).astype(int)

In [9]:
sorted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   index            int64 
 1   event_date       object
 2   event_timestamp  int64 
 3   action_time      int32 
 4   vacancy_id_      int64 
 5   cookie_id        object
 6   user_id          object
 7   event_type       object
dtypes: int32(1), int64(3), object(4)
memory usage: 703.4+ MB


In [ ]:
#отслеживаем смену cookie_id, чтобы обнулить значение между соседними cookie_id
for i, row in sorted_df.iterrows():
    if i == 0:
        sorted_df.iloc[[i], [3]] = 0
    elif sorted_df.iloc[i].cookie_id != sorted_df.iloc[i-1].cookie_id:
        sorted_df.iloc[[i], [3]] = 0

In [15]:
sorted_df = pd.read_parquet('Data/with_action_time.parquet')
sorted_df.head()

,Unnamed: 0,index,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,0,4550690,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,4653413,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,2,4650401,2022-08-30,1661866559,11.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,3,4693679,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,4,4650417,2022-08-30,1661866564,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy


In [16]:
#Удаляем колонку со старым индексом
sorted_df = sorted_df.drop(columns=['index'])

In [17]:
sorted_df

,Unnamed: 0,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,0,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,2,2022-08-30,1661866559,11.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,3,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,4,2022-08-30,1661866564,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
...,...,...,...,...,...,...,...,...
12292583,12292583,2022-08-22,1661164755,123.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292584,12292584,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts
12292585,12292585,2022-08-22,1661165546,791.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292586,12292586,2022-08-22,1661165547,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy


In [18]:
#Сдвигаем солонку action_time на одну ячейку вверх, чтобы в текущей строке в солонке action_time 
#время которое прошло с момента текущего события до следующего события 
sorted_df['action_time'] = sorted_df['action_time'].shift (-1)
sorted_df.iloc[[-1], [2]] = 0
sorted_df

,Unnamed: 0,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,0,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,2,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,3,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,4,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
...,...,...,...,...,...,...,...,...
12292583,12292583,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292584,12292584,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts
12292585,12292585,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292586,12292586,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy


In [19]:
train_mfti_df_up = sorted_df

In [20]:
train_mfti_df_up.count()

Unnamed: 0         12292588
event_date         12292588
event_timestamp    12292588
action_time        12292587
vacancy_id_        12292588
cookie_id          12292588
user_id             8711177
event_type         12292588
dtype: int64

In [21]:
train_mfti_df_up.count()

Unnamed: 0         12292588
event_date         12292588
event_timestamp    12292588
action_time        12292587
vacancy_id_        12292588
cookie_id          12292588
user_id             8711177
event_type         12292588
dtype: int64

In [22]:
train_mfti_df_up.isna()

,Unnamed: 0,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
12292583,False,False,False,False,False,False,True,False
12292584,False,False,False,False,False,False,True,False
12292585,False,False,False,False,False,False,True,False
12292586,False,False,False,False,False,False,True,False


In [23]:
#назначение уникального индекса для каждой пары(куки+юзер) и их дубликатов
#ПОДХОД - всегда смотрим пару куки+юзер
#train_mfti_df_up.insert(loc=0, column='common_id', value=train_mfti_df_up.set_index(['cookie_id','user_id']).index.factorize()[0]+1)

#ПОДХОД - если User Id != NaN, то не ориентируемся на куки, если User Id = NaN, формируем common_id по куки
train_mfti_df_up_only_isna_user=train_mfti_df_up[train_mfti_df_up['user_id'].isna()]
train_mfti_df_up_only_isna_user['common_id']=train_mfti_df_up_only_isna_user.cookie_id.astype(str)
display(train_mfti_df_up_only_isna_user.head())

train_mfti_df_up_only_notna_user=train_mfti_df_up[train_mfti_df_up['user_id'].notna()]
train_mfti_df_up_only_notna_user['common_id']=train_mfti_df_up_only_notna_user.user_id.astype(str)
display(train_mfti_df_up_only_notna_user.head())


train_mfti_df_up=pd.concat([train_mfti_df_up_only_isna_user,train_mfti_df_up_only_notna_user])
train_mfti_df_up

In [24]:
train_mfti_df_up

,common_id,Unnamed: 0,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,1,0,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,1,2,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,1,3,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,1,4,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
...,...,...,...,...,...,...,...,...,...
12292583,367641,12292583,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292584,367641,12292584,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts
12292585,367641,12292585,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292586,367641,12292586,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy


In [25]:
#полный список уникальных пар идентификаторов куки+юзер
v=train_mfti_df_up.groupby(['cookie_id','user_id']).size().reset_index()
lst=v[['cookie_id','user_id']].values.tolist()
#lst

In [26]:
#отображение общего идентификатора на примере конкретного user_id и cookie_id
train_mfti_df_up[(train_mfti_df_up['user_id']=='f5a2326a17484330aa8cb4019f1b1960')&(train_mfti_df_up['cookie_id']=='03bf8c511fa949c79845a5d81b09aa1d')]

,common_id,Unnamed: 0,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
174110,5420,174110,2022-08-01,1659359891,0.0,148714,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
174111,5420,174111,2022-08-01,1659359891,13688.0,148714,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,preview_click_vacancy
174112,5420,174112,2022-08-01,1659373579,0.0,258441,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
174113,5420,174113,2022-08-01,1659373579,39.0,258441,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,preview_click_vacancy
174114,5420,174114,2022-08-01,1659373618,0.0,102914,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...,...,...,...
174193,5420,174193,2022-08-02,1659449607,1.0,126251,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,preview_click_vacancy
174194,5420,174194,2022-08-02,1659449608,5.0,126251,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
174195,5420,174195,2022-08-02,1659449613,1218.0,126251,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
174196,5420,174196,2022-08-02,1659450831,0.0,136081,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,preview_click_vacancy


In [27]:
train_mfti_df_up.to_parquet('dataset_with_common_id.parquet',compression='BROTLI')

In [28]:
train_mfti_df = pd.read_parquet('dataset_with_common_id.parquet')
train_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 9 columns):
 #   Column           Non-Null Count     Dtype  
---  ------           --------------     -----  
 0   common_id        12292588 non-null  int64  
 1   Unnamed: 0       12292588 non-null  int64  
 2   event_date       12292588 non-null  object 
 3   event_timestamp  12292588 non-null  int64  
 4   action_time      12292587 non-null  float64
 5   vacancy_id_      12292588 non-null  int64  
 6   cookie_id        12292588 non-null  object 
 7   user_id          8711177 non-null   object 
 8   event_type       12292588 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 844.1+ MB


In [29]:
# Выводим информацию о количестве событий каждого типа

train_mfti_df['event_type'].value_counts()

show_vacancy              6198889
preview_click_vacancy     4781280
click_response             384090
click_contacts             277584
preview_click_response     190635
click_favorite             155844
preview_click_favorite     107016
preview_click_contacts     102050
click_phone                 79191
preview_click_phone         16009
Name: event_type, dtype: int64

In [30]:
# Выводим информацию о количестве событийв каждый день

train_mfti_df['event_date'].value_counts()

2022-09-05    292929
2022-09-06    288203
2022-09-07    278221
2022-09-19    278194
2022-09-13    276529
               ...  
2022-08-12     98335
2022-08-14     88921
2022-08-13     58818
2022-08-28     15492
2022-08-27     13727
Name: event_date, Length: 62, dtype: int64

In [31]:
# Выводим информацию о количестве уникальных значений для всех колонок

print(f"nUnique vacancy_id_: {train_mfti_df['vacancy_id_'].nunique()}")
print(f"nUnique event_date: {train_mfti_df['event_date'].nunique()}")
print(f"nUnique cookie_id: {train_mfti_df['cookie_id'].nunique()}")
print(f"nUnique user_id: {train_mfti_df['user_id'].nunique()}")

nUnique vacancy_id_: 160167
nUnique event_date: 62
nUnique cookie_id: 330180
nUnique user_id: 209335


In [32]:
# Проверяем, всегда ли одному cookie_id соответствует один user_id.
# НЕТ

train_mfti_df.groupby(['cookie_id'])['user_id'].nunique().value_counts()

1     215167
0     113830
2       1124
3         47
4          5
5          4
56         1
8          1
7          1
Name: user_id, dtype: int64

In [33]:
# Проверяем, всегда ли одному user_id соответствует один cookie_id.
# НЕТ

train_mfti_df.groupby(['user_id'])['cookie_id'].nunique().value_counts()

1     201813
2       6883
3        529
4         81
5         16
6          6
7          4
10         1
8          1
15         1
Name: cookie_id, dtype: int64

In [34]:
train_mfti_df.groupby(['cookie_id'])['event_timestamp'].count()

cookie_id
0000c4548c3944c08972bbdc1fa4eb85    15
0000d7508334414ca792c5ff66eb8c14    24
0000e63f423542fe8090575b611cea4d    31
00013bf3e58244b48d4dbdf43543c30c    11
00015019b0594984afc3fd9fa9557490    81
                                    ..
ffff3581b7e7405db25fac5c5dd1bff9    15
ffff42fe199b46d59ce38bde9c95b148    80
ffff8283d29644e69ad6d08cc3ca7fb7    14
ffffd6d4f2134dd6aa80ff71e942b508    27
ffffdb17f45b4032b386d691d52e6c00    12
Name: event_timestamp, Length: 330180, dtype: int64

**test_public_mfti.parquet** – часть теста, с открытым таргетом, для проверки работоспособности решений.

* cookie_id - id пользователя по его браузеру/ip/устройству
* vacancy_id_ - список вакансий, на которые пользователь откликнулся или позвонил в течение месяца после окончания данных train

In [35]:
test_public_mfti_df = pd.read_parquet('Data_Rabota/test_public_mfti.parquet', engine='pyarrow')
test_public_mfti_df

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [36]:
test_public_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cookie_id    772 non-null    object
 1   vacancy_id_  772 non-null    object
dtypes: object(2)
memory usage: 12.2+ KB


In [37]:
# В train датафрейме
# 1. Группируем события по cookie_id
# 2. Собираем номера вакансий в событиях в список
# 3. Оставляем только уникальные номера вакансий

# Нужно для того, чтобы смерджить с test_public

train_cookie_vac = train_mfti_df.groupby(['cookie_id'])['vacancy_id_'].unique().apply(list).rename('vacancy_id_train')
train_cookie_vac

cookie_id
0000c4548c3944c08972bbdc1fa4eb85     [153975, 182445, 174953, 176171, 137659, 187529]
0000d7508334414ca792c5ff66eb8c14    [192850, 230581, 250327, 177451, 108690, 17383...
0000e63f423542fe8090575b611cea4d    [226832, 122287, 251362, 141592, 158280, 13759...
00013bf3e58244b48d4dbdf43543c30c     [243795, 209629, 135961, 138688, 138502, 250729]
00015019b0594984afc3fd9fa9557490    [242642, 111592, 113482, 110421, 127352, 11150...
                                                          ...                        
ffff3581b7e7405db25fac5c5dd1bff9    [164602, 207423, 102780, 210628, 203404, 23586...
ffff42fe199b46d59ce38bde9c95b148    [105008, 125883, 132245, 220387, 235375, 14731...
ffff8283d29644e69ad6d08cc3ca7fb7             [115949, 127277, 182846, 182877, 231900]
ffffd6d4f2134dd6aa80ff71e942b508    [102931, 155987, 159314, 156033, 171308, 13941...
ffffdb17f45b4032b386d691d52e6c00             [150843, 107893, 107125, 160164, 185412]
Name: vacancy_id_train, Length: 330180, dtyp

In [40]:
# Мерджим по cookie_id и переименовываем колонку для ясности

new_df = test_public_mfti_df.merge(train_cookie_vac, left_on='cookie_id', right_on='cookie_id')
new_df = new_df.rename(columns={'vacancy_id_': 'vacancy_id_public'})
new_df

,cookie_id,vacancy_id_public,vacancy_id_train
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...","[258338, 182454, 258095, 243946, 134006, 10109..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...","[260154, 130142, 250327, 212411, 249949, 21343..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...","[173991, 141447, 179484, 181182, 198114, 24074..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]","[184726, 158242, 252106, 192562, 106598, 18103..."
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]","[107486, 248787, 162242, 167168, 114852, 15021..."
...,...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]","[229314, 243294, 212971, 170333, 103567]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212...","[165335, 181381, 159326, 184411, 104054, 15453..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]","[119075, 195754, 108093, 208350, 250251, 21984..."
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052...","[102794, 236860, 236821, 175053, 101241, 23628..."


In [38]:
# Пишем функцию чтобы проверить, нет ли пересечений в списках вакансий из public и train

def check_intersections(column_1, column_2):
    set_1 = set(column_1)
    set_2 = set(column_2)
    if len(set_1.intersection(set_2)) > 0:
        return 1
    else:
        return 0

In [41]:
check_intersect = new_df.apply(lambda x: check_intersections(x.vacancy_id_public, x.vacancy_id_train), axis=1)
check_intersect

0      0
1      0
2      0
3      0
4      0
      ..
767    0
768    0
769    0
770    0
771    0
Length: 772, dtype: int64

In [42]:
# Пересечений нет

sum(check_intersect)

0

In [43]:
# Убираем из train датасета cookie_id, совпадающие с public

In [44]:
train_mfti_df_fixed = train_mfti_df[~train_mfti_df['cookie_id'].isin(new_df['cookie_id'])]
train_mfti_df_fixed

,common_id,Unnamed: 0,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,1,0,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,1,2,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,1,3,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,1,4,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
...,...,...,...,...,...,...,...,...,...
12292583,367641,12292583,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292584,367641,12292584,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts
12292585,367641,12292585,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292586,367641,12292586,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy


In [20]:
# Создаем колонку с рейтингом действия
# Пока принцип такой: 1 - целевое действие, 0 - остальные действия

purpose_events=['click_response',
                'preview_click_response',
                'click_contacts',
                'preview_click_contacts',
                'click_phone',
                'preview_click_phone']

def check_is_purpose(event):
    if event in purpose_events:
        return 1
    else:
        return 0

train_mfti_df_fixed['rating'] = train_mfti_df_fixed['event_type'].apply(check_is_purpose)
train_mfti_df_fixed

C:\Users\glebychgame\AppData\Local\Temp\ipykernel_6492\3918156442.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mfti_df_fixed['rating'] = train_mfti_df_fixed['event_type'].apply(check_is_purpose)


,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,rating
0,1,2022-08-30,1661866548,0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy,0
1,1,2022-08-30,1661866548,11,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
2,1,2022-08-30,1661866559,0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
3,1,2022-08-30,1661866559,5,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy,0
4,1,2022-08-30,1661866564,0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
...,...,...,...,...,...,...,...,...,...
12292583,367641,2022-08-22,1661164755,0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy,0
12292584,367641,2022-08-22,1661164755,791,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts,1
12292585,367641,2022-08-22,1661165546,1,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy,0
12292586,367641,2022-08-22,1661165547,5,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy,0


In [27]:
train_mfti_df_fixed.to_parquet('last_collected_df.parquet',compression='BROTLI')

In [ ]:
# !!!TODO!!! - Пока группирую пользователей по cookie_id, исправить, когда будет новый user_id

#train_mfti_df_fixed_2 = train_mfti_df_fixed.groupby(['cookie_id', 'vacancy_id_'], as_index=False)['rating'].sum()

# DONE
train_mfti_df_fixed_2 = train_mfti_df_fixed.groupby(['common_id', 'vacancy_id_'], as_index=False)['rating'].sum()
train_mfti_df_fixed_2

In [22]:
train_mfti_df_fixed_2['rating'].value_counts()

0     3961172
1      574208
2       96915
3       33095
4       14144
5        6947
6        3678
7        2051
8        1266
9         780
10        500
11        329
12        248
13        169
14        121
15         90
16         68
17         59
18         40
19         33
20         20
22         16
23         15
27         13
21         11
28          9
24          9
26          9
25          7
30          7
34          6
39          5
33          4
32          4
31          3
36          3
41          3
54          3
29          3
40          2
45          2
52          1
48          1
47          1
62          1
61          1
59          1
68          1
60          1
35          1
57          1
49          1
66          1
93          1
42          1
74          1
79          1
38          1
37          1
Name: rating, dtype: int64

In [23]:
# Сохраняем датасет в .csv

train_mfti_df_fixed_2.to_csv('rabota_test.csv', index=False)

**test_private_users_mfti.parquet** – часть теста, с закрытым таргетом для итоговой проверки решений

* cookie_id - id пользователя по его браузеру/ip/устройству

In [46]:
test_private_users_mfti_df = pd.read_parquet('Data_Rabota/test_private_users_mfti.parquet', engine='pyarrow')
test_private_users_mfti_df

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a
...,...
3081,ffadd195859444d2ade2479b0611c5c1
3082,ffbc08b528c64f22996873fc63872202
3083,ffdeaf3c34544529880aebf17c103f6c
3084,ffefa79a74804ee69e6c131e0d05b948


In [47]:
test_private_users_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cookie_id  3086 non-null   object
dtypes: object(1)
memory usage: 24.2+ KB


In [78]:
train_mfti_df_test = train_mfti_df_fixed[train_mfti_df_fixed['cookie_id'].isin(new_df['cookie_id'])]
train_mfti_df_test

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,rating


**test_private_sample_submission_mfti.parquet** – файл с примером предсказаний, который требуется получить по итогу хакатона 

* cookie_id - id пользователя по его браузеру/ip/устройству
* predictions – список из 5 id вакансий, которые модель предсказала как наиболее релеватные для данного пользователя

In [48]:
test_private_sample_submission_mfti_df = pd.read_parquet('Data_Rabota/test_private_sample_submission_mfti.parquet', engine='pyarrow')
test_private_sample_submission_mfti_df

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [49]:
test_private_sample_submission_mfti_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cookie_id    3086 non-null   object
 1   predictions  3086 non-null   object
dtypes: object(2)
memory usage: 48.3+ KB


#### Добавлены в датасеты test_private_users_mfti_df и test_public_mfti_df  куки и общий идентификатор, пустые user_id учтены так же. 

In [50]:
display(test_private_sample_submission_mfti_df.head())
duplicateRows = test_private_sample_submission_mfti_df[test_private_sample_submission_mfti_df.duplicated(['cookie_id'])]
duplicateRows

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"


,cookie_id,predictions


In [51]:
display(test_private_users_mfti_df.head())
duplicateRows = test_private_users_mfti_df[test_private_users_mfti_df.duplicated(['cookie_id'])]
duplicateRows

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a


,cookie_id


In [52]:
display(test_public_mfti_df.head())
duplicateRows = test_public_mfti_df[test_public_mfti_df.duplicated(['cookie_id'])]
duplicateRows

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"


,cookie_id,vacancy_id_


In [53]:
#добавить юзер и комон идентификаторы в test_private_sample_submission_mfti_df
v=train_mfti_df[['cookie_id','user_id','common_id']].groupby(['cookie_id','common_id','user_id'],dropna=False).size().reset_index()

test_private_sample_submission_mfti_df_up=test_private_sample_submission_mfti_df[['cookie_id','predictions']].merge(v[['cookie_id','user_id','common_id']],how='left',indicator=False)
display(test_private_sample_submission_mfti_df_up.head())
display(test_private_sample_submission_mfti_df_up.info())

#добавить юзер и комон идентификаторы в test_private_users_mfti_df
test_private_users_mfti_df_up=test_private_users_mfti_df[['cookie_id']].merge(v[['cookie_id','user_id','common_id']],how='left',indicator=False)
display(test_private_users_mfti_df_up.head())
display(test_private_users_mfti_df_up.info())

#добавить юзер и комон идентификаторы в test_public_mfti_df
test_public_mfti_df_up=test_public_mfti_df[['cookie_id','vacancy_id_']].merge(v[['cookie_id','user_id','common_id']],how='left',indicator=False)
display(test_public_mfti_df_up.head())
display(test_public_mfti_df_up.info())

,cookie_id,predictions,user_id,common_id
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]",NaN,123
1,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]",1706e3869fc6432b8d7e04c613fefb4d,124
2,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]",535c21153f7f442bb09de85f0d00dbd6,306
3,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]",9e642ae3300e459f8648e573f105fa43,808
4,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]",0c69442f665b4daf8616223323e234a0,832


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3562 entries, 0 to 3561
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cookie_id    3562 non-null   object
 1   predictions  3562 non-null   object
 2   user_id      2678 non-null   object
 3   common_id    3562 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 139.1+ KB


None

,cookie_id,user_id,common_id
0,0018914ba3e54011b28fa715583d3354,NaN,123
1,0018914ba3e54011b28fa715583d3354,1706e3869fc6432b8d7e04c613fefb4d,124
2,0035c298d8c64f368ae730a9cca9bb20,535c21153f7f442bb09de85f0d00dbd6,306
3,00956458877448ec9fba87fb97443fdf,9e642ae3300e459f8648e573f105fa43,808
4,0099387c921b41e7bae6c99dd8254b60,0c69442f665b4daf8616223323e234a0,832


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3562 entries, 0 to 3561
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cookie_id  3562 non-null   object
 1   user_id    2678 non-null   object
 2   common_id  3562 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 111.3+ KB


None

,cookie_id,vacancy_id_,user_id,common_id
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",477e719bfbcc4071a8acc1b357492b00,63
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",753a2a268ae84ab49475e62736c02860,296
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",NaN,910
3,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",d01c76b282364fbb8195f326de3e893c,911
4,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",NaN,1926


<class 'pandas.core.frame.DataFrame'>
Int64Index: 907 entries, 0 to 906
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cookie_id    907 non-null    object
 1   vacancy_id_  907 non-null    object
 2   user_id      665 non-null    object
 3   common_id    907 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 35.4+ KB


None

In [54]:
#сохранить датасеты с сжатием
test_private_sample_submission_mfti_df_up.to_parquet('test_private_sample_submission_mfti_up.parquet',compression='BROTLI')
test_private_users_mfti_df_up.to_parquet('test_private_users_mfti_up.parquet',compression='BROTLI')
test_public_mfti_df_up.to_parquet('test_public_mfti_up.parquet',compression='BROTLI')